# Getting Started
### To start the Jupyter Notebook
* Start the julia program, or the "REPL" (read-eval-print-loop)
* In the Julia REPL, run
    * `Pkg.add("IJulia") # only needed first time`
    * `using IJulia`
    * `notebook()`
* This should open a web-browser where you can create and use Jupyter notebooks
* This window needs to stay open, because it's running a local server (backend) to work with your browser (frontend)

### Install some necessary packages
* This only needs to be run once, but
* This can take quite a while...

In [105]:
if false
    Pkg.add("Plots")          # Plotting Interface
    Pkg.add("PlotlyJS")       # Plotting Backend (good for interactive notebook plots)
    Pkg.add("DSP")            # DSP utilities
    Pkg.add("MAT")            # reading/write of .mat files
    Pkg.add("SampledSignals") # easily listening to vectors
    Pkg.update()
else
    println("NOTE: nothing actually happened\n\n")
end
println("Using Julia version:")
versioninfo()

NOTE: nothing actually happened


Using Julia version:
Julia Version 0.6.2
Commit d386e40c17* (2017-12-13 18:08 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Xeon(R) CPU E5-1650 v4 @ 3.60GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, broadwell)


### Let's start `using` these packages
* This puts convenient functions like `matread`, and `filt` in the ***global namespace***

In [106]:
using DSP
using MAT
using SampledSignals
using Plots
plotlyjs(size=(800,500)) # choose PlotlyJS.jl backend from Plots.jl

Plots.PlotlyJSBackend()

### Create `some_noise` and set `fs`
* and print the rms
* and plot it

In [107]:
fs = 16000
some_noise = rand(50)
println("The RMS of some_noise is $(rms(some_noise))")
plot(some_noise)

The RMS of some_noise is 0.5645188662554577


### Usage notes about `Plots.jl` with `plotlyjs()`
* The plots should be interactive
* To zoom, click + drag the region to zoom in on
* To pan, select the pan icon with up/down/left/right arrows, and click + drag
* To get back to the original view, double click on the plot
* To save a static picture of the plot, click on the camera icon

# Filtering, etc.
### `DSP.jl` has a nice Biquad implementation
The `DSP.jl` documentation is nice and located here, https://dspjl.readthedocs.io/en/stable/

To construct a biquad from `DSP.jl`, we can choose one of the following
* `Biquad(b0, b1, b2, a1, a2)`
* `Biquad(b0, b1, b2, a0, a1, a2)`

Let's make a helper function for creating a single low-pass biquad filter
* The helper function is based on "RBJ" (Robert Bristow-Johnson) material found here
* The entirety of RBJ EQ "cookbook" is hiddent text in this markdown cell :-)

<!--
Cookbook formulae for audio EQ biquad filter coefficients
----------------------------------------------------------------------------
           by Robert Bristow-Johnson  <rbj@audioimagination.com>


All filter transfer functions were derived from analog prototypes (that
are shown below for each EQ filter type) and had been digitized using the
Bilinear Transform.  BLT frequency warping has been taken into account for
both significant frequency relocation (this is the normal "prewarping" that
is necessary when using the BLT) and for bandwidth readjustment (since the
bandwidth is compressed when mapped from analog to digital using the BLT).

First, given a biquad transfer function defined as:

            b0 + b1*z^-1 + b2*z^-2
    H(z) = ------------------------                                  (Eq 1)
            a0 + a1*z^-1 + a2*z^-2

This shows 6 coefficients instead of 5 so, depending on your architechture,
you will likely normalize a0 to be 1 and perhaps also b0 to 1 (and collect
that into an overall gain coefficient).  Then your transfer function would
look like:

            (b0/a0) + (b1/a0)*z^-1 + (b2/a0)*z^-2
    H(z) = ---------------------------------------                   (Eq 2)
               1 + (a1/a0)*z^-1 + (a2/a0)*z^-2

or

                      1 + (b1/b0)*z^-1 + (b2/b0)*z^-2
    H(z) = (b0/a0) * ---------------------------------               (Eq 3)
                      1 + (a1/a0)*z^-1 + (a2/a0)*z^-2


The most straight forward implementation would be the "Direct Form 1"
(Eq 2):

    y[n] = (b0/a0)*x[n] + (b1/a0)*x[n-1] + (b2/a0)*x[n-2]
                        - (a1/a0)*y[n-1] - (a2/a0)*y[n-2]            (Eq 4)

This is probably both the best and the easiest method to implement in the
56K and other fixed-point or floating-point architechtures with a double
wide accumulator.



Begin with these user defined parameters:

    Fs (the sampling frequency)

    f0 ("wherever it's happenin', man."  Center Frequency or
        Corner Frequency, or shelf midpoint frequency, depending
        on which filter type.  The "significant frequency".)

    dBgain (used only for peaking and shelving filters)

    Q (the EE kind of definition, except for peakingEQ in which A*Q is
        the classic EE Q.  That adjustment in definition was made so that
        a boost of N dB followed by a cut of N dB for identical Q and
        f0/Fs results in a precisely flat unity gain filter or "wire".)

     _or_ BW, the bandwidth in octaves (between -3 dB frequencies for BPF
        and notch or between midpoint (dBgain/2) gain frequencies for
        peaking EQ)

     _or_ S, a "shelf slope" parameter (for shelving EQ only).  When S = 1,
        the shelf slope is as steep as it can be and remain monotonically
        increasing or decreasing gain with frequency.  The shelf slope, in
        dB/octave, remains proportional to S for all other values for a
        fixed f0/Fs and dBgain.



Then compute a few intermediate variables:

    A  = sqrt( 10^(dBgain/20) )
       =       10^(dBgain/40)     (for peaking and shelving EQ filters only)

    w0 = 2*pi*f0/Fs

    cos(w0)
    sin(w0)

    alpha = sin(w0)/(2*Q)                                       (case: Q)
          = sin(w0)*sinh( ln(2)/2 * BW * w0/sin(w0) )           (case: BW)
          = sin(w0)/2 * sqrt( (A + 1/A)*(1/S - 1) + 2 )         (case: S)

        FYI: The relationship between bandwidth and Q is
             1/Q = 2*sinh(ln(2)/2*BW*w0/sin(w0))     (digital filter w BLT)
        or   1/Q = 2*sinh(ln(2)/2*BW)             (analog filter prototype)

        The relationship between shelf slope and Q is
             1/Q = sqrt((A + 1/A)*(1/S - 1) + 2)

    2*sqrt(A)*alpha  =  sin(w0) * sqrt( (A^2 + 1)*(1/S - 1) + 2*A )
        is a handy intermediate variable for shelving EQ filters.


Finally, compute the coefficients for whichever filter type you want:
   (The analog prototypes, H(s), are shown for each filter
        type for normalized frequency.)


LPF:        H(s) = 1 / (s^2 + s/Q + 1)

            b0 =  (1 - cos(w0))/2
            b1 =   1 - cos(w0)
            b2 =  (1 - cos(w0))/2
            a0 =   1 + alpha
            a1 =  -2*cos(w0)
            a2 =   1 - alpha



HPF:        H(s) = s^2 / (s^2 + s/Q + 1)

            b0 =  (1 + cos(w0))/2
            b1 = -(1 + cos(w0))
            b2 =  (1 + cos(w0))/2
            a0 =   1 + alpha
            a1 =  -2*cos(w0)
            a2 =   1 - alpha



BPF:        H(s) = s / (s^2 + s/Q + 1)  (constant skirt gain, peak gain = Q)

            b0 =   sin(w0)/2  =   Q*alpha
            b1 =   0
            b2 =  -sin(w0)/2  =  -Q*alpha
            a0 =   1 + alpha
            a1 =  -2*cos(w0)
            a2 =   1 - alpha


BPF:        H(s) = (s/Q) / (s^2 + s/Q + 1)      (constant 0 dB peak gain)

            b0 =   alpha
            b1 =   0
            b2 =  -alpha
            a0 =   1 + alpha
            a1 =  -2*cos(w0)
            a2 =   1 - alpha



notch:      H(s) = (s^2 + 1) / (s^2 + s/Q + 1)

            b0 =   1
            b1 =  -2*cos(w0)
            b2 =   1
            a0 =   1 + alpha
            a1 =  -2*cos(w0)
            a2 =   1 - alpha



APF:        H(s) = (s^2 - s/Q + 1) / (s^2 + s/Q + 1)

            b0 =   1 - alpha
            b1 =  -2*cos(w0)
            b2 =   1 + alpha
            a0 =   1 + alpha
            a1 =  -2*cos(w0)
            a2 =   1 - alpha



peakingEQ:  H(s) = (s^2 + s*(A/Q) + 1) / (s^2 + s/(A*Q) + 1)

            b0 =   1 + alpha*A
            b1 =  -2*cos(w0)
            b2 =   1 - alpha*A
            a0 =   1 + alpha/A
            a1 =  -2*cos(w0)
            a2 =   1 - alpha/A



lowShelf: H(s) = A * (s^2 + (sqrt(A)/Q)*s + A)/(A*s^2 + (sqrt(A)/Q)*s + 1)

            b0 =    A*( (A+1) - (A-1)*cos(w0) + 2*sqrt(A)*alpha )
            b1 =  2*A*( (A-1) - (A+1)*cos(w0)                   )
            b2 =    A*( (A+1) - (A-1)*cos(w0) - 2*sqrt(A)*alpha )
            a0 =        (A+1) + (A-1)*cos(w0) + 2*sqrt(A)*alpha
            a1 =   -2*( (A-1) + (A+1)*cos(w0)                   )
            a2 =        (A+1) + (A-1)*cos(w0) - 2*sqrt(A)*alpha



highShelf: H(s) = A * (A*s^2 + (sqrt(A)/Q)*s + 1)/(s^2 + (sqrt(A)/Q)*s + A)

            b0 =    A*( (A+1) + (A-1)*cos(w0) + 2*sqrt(A)*alpha )
            b1 = -2*A*( (A-1) + (A+1)*cos(w0)                   )
            b2 =    A*( (A+1) + (A-1)*cos(w0) - 2*sqrt(A)*alpha )
            a0 =        (A+1) - (A-1)*cos(w0) + 2*sqrt(A)*alpha
            a1 =    2*( (A-1) - (A+1)*cos(w0)                   )
            a2 =        (A+1) - (A-1)*cos(w0) - 2*sqrt(A)*alpha





FYI:   The bilinear transform (with compensation for frequency warping)
substitutes:

                                  1         1 - z^-1
      (normalized)   s  <--  ----------- * ----------
                              tan(w0/2)     1 + z^-1

   and makes use of these trig identities:

                     sin(w0)                               1 - cos(w0)
      tan(w0/2) = -------------           (tan(w0/2))^2 = -------------
                   1 + cos(w0)                             1 + cos(w0)


   resulting in these substitutions:


                 1 + cos(w0)     1 + 2*z^-1 + z^-2
      1    <--  ------------- * -------------------
                 1 + cos(w0)     1 + 2*z^-1 + z^-2


                 1 + cos(w0)     1 - z^-1
      s    <--  ------------- * ----------
                   sin(w0)       1 + z^-1

                                      1 + cos(w0)     1         -  z^-2
                                  =  ------------- * -------------------
                                        sin(w0)       1 + 2*z^-1 + z^-2


                 1 + cos(w0)     1 - 2*z^-1 + z^-2
      s^2  <--  ------------- * -------------------
                 1 - cos(w0)     1 + 2*z^-1 + z^-2


   The factor:

                    1 + cos(w0)
                -------------------
                 1 + 2*z^-1 + z^-2

   is common to all terms in both numerator and denominator, can be factored
   out, and thus be left out in the substitutions above resulting in:


                 1 + 2*z^-1 + z^-2
      1    <--  -------------------
                    1 + cos(w0)


                 1         -  z^-2
      s    <--  -------------------
                      sin(w0)


                 1 - 2*z^-1 + z^-2
      s^2  <--  -------------------
                    1 - cos(w0)


   In addition, all terms, numerator and denominator, can be multiplied by a
   common (sin(w0))^2 factor, finally resulting in these substitutions:


      1         <--   (1 + 2*z^-1 + z^-2) * (1 - cos(w0))

      s         <--   (1         -  z^-2) * sin(w0)

      s^2       <--   (1 - 2*z^-1 + z^-2) * (1 + cos(w0))

      1 + s^2   <--   2 * (1 - 2*cos(w0)*z^-1 + z^-2)


   The biquad coefficient formulae above come out after a little
   simplification.
-->



In [108]:
function BQLPF(f0, fs, Q)
    w0 = float(2*pi*f0/fs)
    alpha = sin(w0)/(2*Q)
    
    
    b0 =  (1 - cos(w0))/2
    b1 =   1 - cos(w0)
    b2 =  (1 - cos(w0))/2
    a0 =   1 + alpha
    a1 =  -2*cos(w0)
    a2 =   1 - alpha
    Biquad(float.([b0, b1, b2, a0, a1, a2])...)
end


BQLPF (generic function with 1 method)

### Syntax notes on BQLPF
1. Julia functions automatically return the output of the last expression
    * So it's actually returning the Biquad object, even though there's no return statement
2. `[b0, b1, b2, a0, a1, a2]` creates an Array with those elements, similar to MATLAB
    * In Python, this makes a **`list`**, which is different from an **`ndarray`** from NumPy
    * In Julia, the equivalent of a Python list is a 1-D Array of type `Any` or **`Array{Any,1}`**
    * **`Array{Any,1}()`** will produce an Array where 
        * `eltype = Any`
        * `ndims = 1`
        * Initial `length = 0`
        * elements can be appended or `push!`
3. In Julia, `float.(someArray)` returns an Array where `float` has been called on every element
    * Similar to `a .* b`, the ***dot operator*** just means "do this operation for every element"
4. `Biquad` is expecting 5 or 6 numbers, not an Array.  To present all elements of an array as an indvidual function arguments, we use the 'splat' operator, which is `...` AFTER some array

### Let's use the helper function to create a low-pass-filter biquad

In [109]:
f0 = 500
Qfac = 2
myBQ = BQLPF(f0, fs, Qfac)

DSP.Filters.Biquad{Float64}(0.009160574920606297, 0.018321149841212594, 0.009160574920606297, -1.8703488223002027, 0.9069911219826279)

### Plot `some_noise` vs. `filt(myBQ, some_noise)`

In [110]:
p = plot(some_noise, lw=3) # create initial plot object
plot!(p, filt(myBQ, some_noise), lw=3) # modify original plot object

### We've definitely filtered out some higher frequencies!
* But what's the frequency response of the `Biquad` we just created?

In [111]:
flist = linspace(0, 0.99*fs/2, 500)
h = freqz(myBQ, flist, fs)
p = plot(flist, abs.(h), lw=3)
title!(p, "linear response")
ylabel!(p, "dB")
xlabel!(p, "Hz")

In [112]:
p = plot(flist, 20*log10.(abs.(h)), lw=3); 
title!(p, "dB response")
ylabel!(p, "dB")
xlabel!(p, "Hz")

In [113]:
p = plot(flist, angle.(h), lw=3); 
title!(p, "phase response")
ylabel!(p, "radians")
xlabel!(p, "Hz")

### Syntax notes on plotting and the **`!`** (bang) operator
* Julia uses the ***convention*** that a function ending with a 'bang' or **`!`** will modify the first variable
    * So **`p = plot(rand(50))`** creates the plot, but **`plot!(p, randn(50))`** adds another line to the plot, **`p`**
* I stress that this is a ***convention*** because it's not enforced
    * That said, it's considered good practice to put a **`!`** at the end of a function name that modifies the input

# What about audio?
For convenience, we'll use the excellent SampledSignals package to play audio samples

Let's make a longer noise sequence, use our biquad filter, and then listen to the "before" and "after"

In [116]:
new_noise = rand(Int(fs*5)) - 0.5
SampleBuf(new_noise, fs)

80000-frame, 1-channel SampleBuf{Float64, 1}
5.0s sampled at 16000.0Hz
▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇

### Filtering the `new_noise`
* The difference in spectrum should be obvious between these two examples

In [117]:
SampleBuf(filt(myBQ, new_noise), fs)

80000-frame, 1-channel SampleBuf{Float64, 1}
5.0s sampled at 16000.0Hz
▇▆▇▇▇▇▆▇▆▇▆▆▆▆▆▆▆▆▆▆▆▇▆▆▆▆▇▇▆▆▇▆▆▇▇▆▆▇▆▇▇▇▇▇▇▇▇▆▆▆▆▆▇▇▆▇▆▇▇▆▆▇▇▇▆▇▇▆▆▇▇▇▆▇▆▆▇▇▆▆